<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/LlamaIndex/4-Streaming01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 패키지 설치
!pip install llama-index-core llama-index-utils-workflow llama-index-llms-openai llama-parse llama-index-embeddings-openai llama-index-readers-whisper llama-index-tools-yahoo-finance
!pip install llama-index-tools-tavily-research

In [4]:
# 패키지 Import
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.tools.tavily_research import TavilyToolSpec
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCallResult,
    AgentStream,
)
import os
import asyncio

# # 환경변수 설정 (API 키를 여기에 직접 입력해주세요)
# os.environ["OPENAI_API_KEY"] = "여기에_OPENAI_API_KEY_입력"
# os.environ["TAVILY_API_KEY"] = "여기에_TAVILY_API_KEY_입력"

In [5]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [6]:
# 언어 모델 설정
llm = OpenAI(model="gpt-4o-mini")

# Tavily 검색 도구 설정
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))

# 워크플로우 에이전트 설정
workflow = AgentWorkflow.from_tools_or_functions(
    tavily_tool.to_tool_list(),
    llm=llm,
    system_prompt="You're a helpful assistant that can search the web for information."
)

In [7]:
# 비동기 실행 함수
async def main():
    handler = workflow.run(user_msg="What's the weather like in San Francisco?")

    # 스트리밍 결과 처리
    async for event in handler.stream_events():
        if isinstance(event, AgentStream):
            print(event.delta, end="", flush=True)
        elif isinstance(event, AgentInput):
            print("Agent input:", event.input)
            print("Agent name:", event.current_agent_name)
        elif isinstance(event, AgentOutput):
            print("\nAgent output:", event.response)
            print("Tool calls made:", event.tool_calls)
            print("Raw LLM response:", event.raw)
        elif isinstance(event, ToolCallResult):
            print("\nTool called:", event.tool_name)
            print("Arguments to the tool:", event.tool_kwargs)
            print("Tool output:", event.tool_output)

    # 최종 결과 출력
    final_result = await handler
    print("\nFinal result:", final_result)

In [9]:
import nest_asyncio

# Colab 환경에서 비동기 함수 실행
nest_asyncio.apply()
asyncio.run(main())

Agent input: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="You're a helpful assistant that can search the web for information.")]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="What's the weather like in San Francisco?")])]
Agent name: Agent

Agent output: assistant: None
Tool calls made: [ToolSelection(tool_id='call_3o6X795hj31in24uYNzCtNiZ', tool_name='search', tool_kwargs={'query': 'current weather in San Francisco'})]
Raw LLM response: {'id': 'chatcmpl-B8Lhgq4yCdQucxD56sEFzBLIp6veK', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None}], 'created': 1741329752, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_06737a9306', 'usage': None}

Tool called: search
